In [40]:
import pandas as pd
import numpy as np
import joblib
import os
from utils.data_processing import read_all_features, read_supplementary_features, feature_concat
from sklearn.feature_selection import VarianceThreshold
from models.regression_model import RegressionModel
import warnings
warnings.filterwarnings('ignore')

# 配置参数
SEED = 0  # 使用的模型对应的seed
MAIN_FILE_PATH = f"predict_data/47-imagemol-Finturn.csv"  # 新数据主文件路径
SUPP_FILE_PATHS = [
    f"predict_data/47-Morder.csv",
    #f"predict_data/47-QC.csv",
    f"predict_data/47-cddd.csv"

]  # 新数据补充文件路径
MODEL_PATH = f"saved_models/model_seed{SEED}.pkl"  # 模型路径
OUTPUT_PATH = "predictions/predictions_new_data.csv"  # 输出路径
SCALED_OUTPUT_PATH = "predictions/predictions_new_data_scaled.csv"  # 缩放后输出路径
FEATURE_THRESHOLD = 0.01  # 特征选择阈值（与训练时一致）

TARGET_MIN = 10  # 目标最小合理值
TARGET_MAX = 525  # 目标最大合理值

# 确保输出目录存在
output_dir = os.path.dirname(OUTPUT_PATH)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    print(f"创建输出目录: {output_dir}")

print(f"\n{'='*50}")
print(f"加载模型: {MODEL_PATH}")
model = joblib.load(MODEL_PATH)
print(f"模型加载成功!")

print("\n模型属性检查:")
print(f"是否有特征选择器: {'是' if hasattr(model, 'var_selector') else '否'}")
print(f"是否有特征标准化器: {'是' if hasattr(model, 'feature_scaler') else '否'}")
print(f"是否有标签标准化器: {'是' if hasattr(model, 'label_scaler') else '否'}")

expected_features = model.feature_scaler.n_features_in_
print(f"训练时的特征数量: {expected_features}")

print(f"\n{'='*50}")
print(f"读取并处理新数据...")

print(f"\n读取主特征文件: {MAIN_FILE_PATH}")
main_data = pd.read_csv(MAIN_FILE_PATH)
print(f"主特征文件形状: {main_data.shape}")
print(f"主特征文件列数: {len(main_data.columns)}")
print(f"主特征文件数据类型: {main_data.dtypes.value_counts()}")

supp_data_list = []
for file_path in SUPP_FILE_PATHS:
    print(f"\n读取补充特征文件: {file_path}")
    supp_data = pd.read_csv(file_path)
    print(f"补充特征文件形状: {supp_data.shape}")
    print(f"补充特征文件列数: {len(supp_data.columns)}")
    print(f"补充特征文件数据类型: {supp_data.dtypes.value_counts()}")
    supp_data_list.append(supp_data)

print(f"\n拼接特征...")
train_x_new, test_x_new, _ = read_all_features([MAIN_FILE_PATH])
train_supp_new, test_supp_new, _ = read_supplementary_features(SUPP_FILE_PATHS)
test_all_new, _ = feature_concat(test_x_new, test_supp_new, pd.DataFrame(), pd.DataFrame())
print(f"使用feature_concat拼接后的特征形状: {test_all_new.shape}")

all_features_direct = pd.concat([main_data] + supp_data_list, axis=1)
print(f"直接拼接后的特征形状: {all_features_direct.shape}")

print(f"\n{'='*50}")
print(f"特征处理...")

def process_non_numeric_columns(df):
    """处理DataFrame中的非数值列"""
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns
    if len(non_numeric_cols) > 0:
        print(f"发现非数值列: {non_numeric_cols.tolist()}")
        print("这些列将被移除，因为它们无法用于数值特征选择")
        df = df.drop(non_numeric_cols, axis=1)
    return df

processed_features = process_non_numeric_columns(test_all_new)
print(f"处理非数值列后的特征形状: {processed_features.shape}")

if processed_features.empty:
    raise ValueError("处理后没有剩余特征，请检查数据格式")

try:
    # 优先使用模型中保存的特征选择器
    if hasattr(model, 'var_selector'):
        selected_features = model.var_selector.transform(processed_features)
        print("使用模型中保存的var_selector进行特征选择")
        print(f"特征选择后的形状: {selected_features.shape}")

        # 验证特征选择器
        if hasattr(model.var_selector, 'get_support'):
            mask = model.var_selector.get_support()
            print(f"特征选择器保留的特征比例: {mask.sum()}/{len(mask)} = {mask.sum()/len(mask):.2%}")
    else:
        # 尝试加载单独保存的特征选择器
        selector_path = f"saved_models/var_selector_seed{SEED}.pkl"
        if os.path.exists(selector_path):
            var_selector = joblib.load(selector_path)
            selected_features = var_selector.transform(processed_features)
            print(f"从 {selector_path} 加载特征选择器")
            print(f"特征选择后的形状: {selected_features.shape}")
        else:
            raise AttributeError("模型中未找到var_selector，且单独保存的选择器也不存在")
except AttributeError:
    print(f"未找到保存的特征选择器，使用阈值{FEATURE_THRESHOLD}初始化新的特征选择器")
    var_selector = VarianceThreshold(threshold=FEATURE_THRESHOLD)
    selected_features = var_selector.fit_transform(processed_features)
    print("使用了新的特征选择器，可能与训练时的特征选择不一致")
    print(f"特征选择后的形状: {selected_features.shape}")

print(f"\n{'='*50}")
print(f"特征标准化...")

# 检查特征数量是否匹配
actual_features = selected_features.shape[1]
if actual_features != expected_features:
    print(f"特征数量不匹配！期望 {expected_features} 个特征，但得到 {actual_features} 个特征")

    # 尝试使用模型中的特征选择器获取特征掩码
    if hasattr(model, 'var_selector'):
        print("尝试使用模型中的特征选择器获取特征掩码...")
        try:
            # 获取特征掩码
            mask = model.var_selector.get_support()

            # 确保processed_features是DataFrame
            if not isinstance(processed_features, pd.DataFrame):
                processed_features = pd.DataFrame(processed_features)

            # 重新选择特征
            selected_features = processed_features.iloc[:, mask]
            print(f"调整后的特征形状: {selected_features.shape}")

            # 再次检查特征数量
            actual_features = selected_features.shape[1]
            if actual_features != expected_features:
                print(f"调整后特征数量仍不匹配！期望 {expected_features} 个特征，但得到 {actual_features} 个特征")
                print("请检查数据格式和特征处理流程是否与训练时一致")
        except Exception as e:
            print(f"获取特征掩码时出错: {e}")
    else:
        print("模型中没有保存特征选择器，无法获取特征掩码")
        print("请确保数据格式和特征处理流程与训练时一致")

# 特征标准化
try:
    # 检查特征标准化器
    print(f"\n特征标准化器参数:")
    if hasattr(model.feature_scaler, 'mean_'):
        print(f"均值: {model.feature_scaler.mean_[:5]}... (仅显示前5个)")
        print(f"方差: {model.feature_scaler.var_[:5]}...")
    elif hasattr(model.feature_scaler, 'min_'):
        print(f"最小值: {model.feature_scaler.min_[:5]}...")
        print(f"数据范围: {model.feature_scaler.scale_[:5]}...")
    else:
        print("特征标准化器类型未知")

    # 检查原始特征的统计信息
    if isinstance(selected_features, pd.DataFrame):
        print(f"\n原始特征统计信息:")
        print(selected_features.describe().loc[['mean', 'std', 'min', 'max']].head())

    # 应用标准化
    scaled_features = model.feature_scaler.transform(selected_features)
    print("特征标准化成功")

    # 检查标准化后的特征统计信息
    print(f"\n标准化后特征统计信息:")
    print(f"均值: {scaled_features.mean(axis=0)[:5]}...")
    print(f"标准差: {scaled_features.std(axis=0)[:5]}...")

except ValueError as e:
    print(f"特征标准化失败: {e}")
    print("请检查数据格式和特征处理流程是否与训练时一致")
    raise

print(f"\n{'='*50}")
print(f"模型预测...")

# 执行预测（输出为归一化值，需反归一化）
prediction_normalized = model.predict(scaled_features)
print(f"归一化预测值范围: [{prediction_normalized.min():.4f}, {prediction_normalized.max():.4f}]")
print(f"归一化预测值均值: {prediction_normalized.mean():.4f}")
print(f"归一化预测值标准差: {prediction_normalized.std():.4f}")

# 通过反归一化获得原始尺度的预测值
try:
    # 检查标签标准化器类型
    label_scaler_type = type(model.label_scaler).__name__
    print(f"\n标签标准化器类型: {label_scaler_type}")

    # 根据标准化器类型输出不同的参数信息
    if label_scaler_type == 'StandardScaler':
        print(f"均值: {model.label_scaler.mean_[0]:.4f}")
        print(f"方差: {model.label_scaler.var_[0]:.4f}")
    elif label_scaler_type == 'MinMaxScaler':
        print(f"最小值: {model.label_scaler.min_[0]:.4f}")
        print(f"数据范围: {model.label_scaler.scale_[0]:.4f}")
        print(f"原始数据范围: [{model.label_scaler.data_min_[0]:.4f}, {model.label_scaler.data_max_[0]:.4f}]")
    else:
        print(f"未知的标签标准化器类型: {label_scaler_type}")

    # 执行反归一化
    prediction_original = model.label_scaler.inverse_transform(
        prediction_normalized.reshape(-1, 1)).flatten()

    print(f"\n原始尺度预测值范围: [{prediction_original.min():.4f}, {prediction_original.max():.4f}]")
    print(f"原始尺度预测值均值: {prediction_original.mean():.4f}")
    print(f"原始尺度预测值标准差: {prediction_original.std():.4f}")

    # 检查预测值是否在合理范围内
    if prediction_original.max() > 10000 or prediction_original.min() < -10000:
        print(f"警告：预测值超出合理范围，可能存在问题！")
        print(f"请检查特征处理流程和标签标准化器是否正确")

except Exception as e:
    print(f"标签反归一化失败: {e}")
    print("请检查标签标准化器是否正确")
    raise

print(f"\n{'='*50}")
print(f"保存预测结果...")

# 读取原始数据以获取样本ID
all_data_new = pd.read_csv(MAIN_FILE_PATH)

# 生成结果DataFrame
results_df = pd.DataFrame({
    'Sample_ID': all_data_new.index,  # 使用索引作为样本ID
    'Predicted_Value': prediction_original
})

# 保存原始预测结果
#results_df.to_csv(OUTPUT_PATH, index=False)
#print(f" 原始预测结果已保存至: {OUTPUT_PATH}")
print(f"原始预测结果统计:\n{results_df['Predicted_Value'].describe()}")


print(f"\n{'='*50}")
print(f"缩放处理预测结果...")

# 检查原始预测结果的范围
current_min = prediction_original.min()
current_max = prediction_original.max()

# 定义缩放函数
def scale_predictions(predictions, current_min, current_max, target_min, target_max):
    """将预测值缩放到目标范围"""
    # 处理当前范围为零的情况（防止除以零）
    if current_max == current_min:
        print("警告：当前预测值范围为零，无法进行线性缩放")
        return np.ones_like(predictions) * (target_min + target_max) / 2

    # 线性缩放
    scaled = (predictions - current_min) / (current_max - current_min) * (target_max - target_min) + target_min
    return scaled

# 应用缩放
print(f"当前预测值范围: [{current_min:.4f}, {current_max:.4f}]")
print(f"目标缩放范围: [{TARGET_MIN}, {TARGET_MAX}]")

# 对预测值进行缩放
scaled_predictions = scale_predictions(
    prediction_original,
    current_min,
    current_max,
    TARGET_MIN,
    TARGET_MAX
)

# 创建缩放后的结果DataFrame
scaled_results_df = pd.DataFrame({
    'Sample_ID': all_data_new.index,  # 使用索引作为样本ID
    'Predicted_Value': scaled_predictions
})

# 保存缩放后的预测结果
scaled_results_df.to_csv(SCALED_OUTPUT_PATH, index=False)
print(f"缩放后的预测结果已保存至: {SCALED_OUTPUT_PATH}")
print(f"缩放后预测结果统计:\n{scaled_results_df['Predicted_Value'].describe()}")

# 保存预测结果的详细统计
stats_file = OUTPUT_PATH.replace('.csv', '_stats.txt')
with open(stats_file, 'w') as f:
    f.write(f"原始预测结果统计:\n{results_df['Predicted_Value'].describe()}\n")
    f.write(f"\n缩放后预测结果统计:\n{scaled_results_df['Predicted_Value'].describe()}\n")
    f.write(f"\n归一化预测值统计:\n")
    f.write(f"范围: [{prediction_normalized.min():.4f}, {prediction_normalized.max():.4f}]\n")
    f.write(f"均值: {prediction_normalized.mean():.4f}\n")
    f.write(f"标准差: {prediction_normalized.std():.4f}\n")
    f.write(f"\n原始尺度预测值统计:\n")
    f.write(f"范围: [{prediction_original.min():.4f}, {prediction_original.max():.4f}]\n")
    f.write(f"均值: {prediction_original.mean():.4f}\n")
    f.write(f"标准差: {prediction_original.std():.4f}\n")
    f.write(f"\n标签标准化器类型: {label_scaler_type}\n")
    if label_scaler_type == 'StandardScaler':
        f.write(f"均值: {model.label_scaler.mean_[0]:.4f}\n")
        f.write(f"方差: {model.label_scaler.var_[0]:.4f}\n")
    elif label_scaler_type == 'MinMaxScaler':
        f.write(f"最小值: {model.label_scaler.min_[0]:.4f}\n")
        f.write(f"数据范围: {model.label_scaler.scale_[0]:.4f}\n")
        f.write(f"原始数据范围: [{model.label_scaler.data_min_[0]:.4f}, {model.label_scaler.data_max_[0]:.4f}]\n")
    f.write(f"\n缩放参数:\n")
    f.write(f"当前范围: [{current_min:.4f}, {current_max:.4f}]\n")
    f.write(f"目标范围: [{TARGET_MIN}, {TARGET_MAX}]\n")
print(f"预测统计信息已保存至: {stats_file}")



加载模型: saved_models/model_seed0.pkl
模型加载成功!

模型属性检查:
是否有特征选择器: 是
是否有特征标准化器: 是
是否有标签标准化器: 是
训练时的特征数量: 2063

读取并处理新数据...

读取主特征文件: predict_data/47-imagemol-Finturn.csv
主特征文件形状: (47, 515)
主特征文件列数: 515
主特征文件数据类型: float64    510
int64        3
object       2
Name: count, dtype: int64

读取补充特征文件: predict_data/47-Morder.csv
补充特征文件形状: (47, 1616)
补充特征文件列数: 1616
补充特征文件数据类型: float64    1055
int64       559
object        2
Name: count, dtype: int64

读取补充特征文件: predict_data/47-cddd.csv
补充特征文件形状: (47, 515)
补充特征文件列数: 515
补充特征文件数据类型: float64    512
object       2
int64        1
Name: count, dtype: int64

拼接特征...
读取的文件 predict_data/47-imagemol-Finturn.csv 的形状: (47, 515)
训练集的形状: (0, 515), 测试集的形状: (47, 515)
从文件 predict_data/47-imagemol-Finturn.csv 提取的训练特征形状: (0, 512), 测试特征形状: (47, 512)
拼接后的训练特征形状: (0, 512), 测试特征形状: (47, 512)
读取的文件 predict_data/47-Morder.csv 的形状: (47, 1616)
训练集的形状: (0, 1616), 测试集的形状: (47, 1616)
从补充文件 predict_data/47-Morder.csv 提取的训练特征形状: (0, 1614), 测试特征形状: (47, 1614)
读取的文件 predict_data/47-c